In [270]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [271]:
# !pip install ta-lib

## import

In [6]:
import numpy as np
import pandas as pd
from kite_trade import *
import os
# import ta

import talib
import datetime
import pyarrow.parquet as pq
import pyarrow as pa

import multiprocessing
import time
from tqdm import tqdm
import util_fun as uf

from pandas.tseries.offsets import MonthEnd
from datetime import timedelta

import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import mplfinance as mpf


from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA


In [7]:
enctoken = "GfeaerpAL6S6AWGUUSV1rve6Sw7axfiGrEJmDncKRQK13Uwj0gA8jcOngCU1/zEyiurcDcp/hsd9X0LhkRdCpjLvotmeWBnaIyLVZSHKALu+fhczoRzw4A=="
kite = KiteApp(enctoken=enctoken)

In [8]:
timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
# inst_filter.query("Symbol. 'NIFTY'")
inst_filter[inst_filter.Symbol.str.contains('HDFC')]


,instrument_token,exchange_token,Symbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
3076,340481,1330,HDFC,HDFC,0.0,None,0.0,0.05,1,EQ,NSE,NSE
3078,2709505,10584,HDFC50INAV,HDFCNEXT50 INAV,0.0,None,0.0,0.01,1,EQ,NSE,NSE
3079,1086465,4244,HDFCAMC,HDFC AMC,0.0,None,0.0,0.05,1,EQ,NSE,NSE
3080,341249,1333,HDFCBANK,HDFC BANK,0.0,None,0.0,0.05,1,EQ,NSE,NSE
3081,2884097,11266,HDFCGRINAV,HDFCAMC - HDFCGRINAV,0.0,None,0.0,0.01,1,EQ,NSE,NSE
3082,2877697,11241,HDFCGROWTH,HDFCAMC - HDFCGROWTH,0.0,None,0.0,0.01,1,EQ,NSE,NSE
3083,119553,467,HDFCLIFE,HDFC LIFE INS CO,0.0,None,0.0,0.05,1,EQ,NSE,NSE
3084,2956033,11547,HDFCLOWVOL,HDFCAMC - HDFCLOWVOL,0.0,None,0.0,0.01,1,EQ,NSE,NSE
3085,2957569,11553,HDFCLVINAV,HDFCAMC - HDFCLOWVOL,0.0,None,0.0,0.01,1,EQ,NSE,NSE
3086,5003009,19543,HDFCMFGETF,HDFC GOLD ETF,0.0,None,0.0,0.01,1,EQ,NSE,NSE


In [275]:

# nifty_500 = pd.read_csv('../data/ind_nifty500list.csv')
# nifty_500 = nifty_500.merge(inst_filter, on = 'Symbol')
# inst_dict = dict(zip(nifty_500.Symbol, nifty_500.instrument_token))

# start_dt = '2018-01-01'
# end_dt = '2023-02-02'
# time_frame = 'day'
# for symbol, instument in tqdm(inst_dict.items()):
#     print(symbol, instument)
#     df_day = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
#     directory = f'../data/historical/{time_frame}/{symbol}'
#     if not os.path.exists(directory):
#         os.makedirs(directory)
#     df_day.to_parquet(f'{directory}/part0.parquet')
#     # break
          

In [276]:
df_day = df_day.between_time('09:15', '03:25')
df_day = df_day[df_day.Volume != 0]
df_day['ema_5'] = talib.EMA(df_day['Close'], timeperiod = 5)
df_day.dropna(inplace=True)
df_day['above_5_ema'] = df_day['Low'] > df_day['ema_5']


In [277]:
# df_day = df_day[df_day.index.strftime('%Y-%m-%d').isin(["2023-01-18","2023-01-19"]) ].copy()

## Backtesting 

## 5 ema

In [278]:
# 5 ema above is my signal candle
# if  next candle is signal  dont do anything
# if next candle is 

# if current one is crossing previous candle low and previous candle is above 5 ema

In [279]:
def prev_ret(data):
    # print(data)
    return data

In [280]:
class RsiOscilator(Strategy):
    back = 5
    rtr = 2
    def init(self):
        self.ema_5 = self.I(talib.EMA, self.data.Close, self.back)
        self.previous_low = self.I(prev_ret,self.data.Low,plot=False)
        self.previous_high = self.I(prev_ret,self.data.High,plot=False)
    
    def next(self):
        if  (self.previous_low[-2] > self.ema_5[-2]) and (self.data.Close[-1] < self.previous_low[-2]):
            stop_loss = max(self.data.High[-1:-4:-1])
            target = (stop_loss - self.data.Close[-1]) * self.rtr
            # print(f'stop_loss, target', stop_loss,self.data.Close[-1] - target)
            self.sell(sl =stop_loss, tp =self.data.Close[-1] - target  ) 
        # elif self.data.High[-1]< self.ema_5:    
        #     self.position.close()

bt = Backtest(df_day,RsiOscilator, cash = 100000)

In [281]:
stats = bt.optimize(
    back = range(3,10,1),
    rtr = range(1,3,1)    
)
stats

  0%|          | 0/14 [00:00<?, ?it/s]

Start                     2018-01-11 00:00...
End                       2023-01-20 00:00...
Duration                   1835 days 00:00:00
Exposure Time [%]                   18.373729
Equity Final [$]                    106100.95
Equity Peak [$]                     109628.35
Return [%]                            6.10095
Buy & Hold Return [%]              302.977359
Return (Ann.) [%]                    1.205899
Volatility (Ann.) [%]                9.016334
Sharpe Ratio                         0.133746
Sortino Ratio                        0.190642
Calmar Ratio                         0.039111
Max. Drawdown [%]                  -30.832557
Avg. Drawdown [%]                   -6.523691
Max. Drawdown Duration     1352 days 00:00:00
Avg. Drawdown Duration      269 days 00:00:00
# Trades                                   19
Win Rate [%]                        36.842105
Best Trade [%]                      17.994648
Worst Trade [%]                     -5.519527
Avg. Trade [%]                    

In [282]:
stats._strategy

<Strategy RsiOscilator(back=3,rtr=2)>

In [283]:
# stats = bt.run()
# stats

In [284]:
# stats._trades

In [285]:
bt.plot()

Row(id='14751', ...)

## vwape

In [286]:
after 10.45:
	if price below vwap:
		Srsi below is below red 
		it it breaks the next below pivot 
			Sell -> stop loss above vwap and target 1%
	if price above vwap:
		srsi above the red
		if it breaks the next above pivot :
			buy stop loss above vwap and target 1%
  

SyntaxError: invalid syntax (3291927124.py, line 1)

In [ ]:
start_date = '2023-01-01'
end_data = '2023-01-20'
symbol = 'HDFCBANK'
df_day = uf.get_instrument_data(
            kite,
            inst_filter,
            timerframe_list = timerframe_list[3:4],
            start_date =start_date ,
            end_date = end_data,
            symbol = symbol)
            

df_day = df_day.between_time('10:45', '03:25')
df_day = df_day[df_day.Volume != 0]
# df_day['ema_5'] = talib.EMA(df_day['Close'], timeperiod = 5)
# df_day.dropna(inplace=True)
# df_day['above_5_ema'] = df_day['Low'] > df_day['ema_5']


2023-01-01 2023-01-20
5minute


100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


In [ ]:
import ta

In [ ]:
# ta.volume.VolumeWeightedAveragePrice(df_day['High'], df_day['Low'], df_day['Close'], df_day['Volume'])

# nifty future intraday streategy 3rd candle

In [500]:
start_dt = '2020-05-01'
end_dt = '2023-02-02'
time_frame = '5minute'
instument = 256265
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/33 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/33 [00:00<?, ?it/s]

In [501]:
df_day_fetch.head()

,Open,High,Low,Close,Volume
Date,,,,,
2020-05-04 00:00:00+05:30,9533.50,9533.50,9266.95,9293.50,0
2020-05-05 00:00:00+05:30,9429.40,9450.90,9190.75,9205.60,0
2020-05-06 00:00:00+05:30,9226.79,9346.90,9116.50,9270.90,0
2020-05-07 00:00:00+05:30,9234.04,9277.85,9175.90,9199.04,0
2020-05-08 00:00:00+05:30,9376.95,9382.65,9238.20,9251.50,0


In [524]:
def get_day_pivot_added(df_day_fetch, df_min_5):
    df_day_fetch_temp = df_day_fetch.copy()
    
    #calculating pivot
    df_day_fetch['pivot'] = (df_day_fetch['High'] + df_day_fetch['Low'] + df_day_fetch['Close'])/3
    df_day_fetch['bottom_pivot'] =  (df_day_fetch['High'] + df_day_fetch['Low'])/2
    df_day_fetch['top_pivot'] = (df_day_fetch['pivot'] - df_day_fetch['bottom_pivot']) + df_day_fetch['pivot']
    df_day_fetch = df_day_fetch.shift(1)

    # adding preiouse day price 
    df_day_fetch.rename(
        columns = {'Close': "prev_close", 'High':"prev_high","Low": "prev_low",
        "Open":'prev_open'}, inplace = True)
    df_day_fetch = df_day_fetch_temp.merge(df_day_fetch,left_index=True, right_index=True)

    # merging pivot with 5 min candle
    df_min_5['date_only'] = pd.to_datetime(df_min_5.index.date)
    df_day_fetch['date_only'] = pd.to_datetime(df_day_fetch.index.date)
    df_day_fetch.rename(
        columns = {'Close': "day_close", 'High':"day_high","Low": "day_low",
        "Open":'day_open'}, inplace = True)
    df_min_5.reset_index(inplace = True)
    df_min_5 = df_min_5.merge(df_day_fetch,how = 'left', on = 'date_only')
    df_min_5.set_index('Date',inplace = True)
    df_min_5.dropna(inplace =True)
    df_min_5.drop(['Volume_x','Volume_y'],inplace = True,axis =1 )
    
    return df_min_5


In [525]:
df_day_fetch = temp1.copy()
df_5min = temp2.copy()
df_5min = get_day_pivot_added(df_day_fetch, df_5min)


In [526]:
df_5min.head()

,Open,High,Low,Close,Volume,date_only,day_open,day_high,day_low,day_close,prev_open,prev_high,prev_low,prev_close,pivot,bottom_pivot,top_pivot
Date,,,,,,,,,,,,,,,,,
2020-05-05 09:15:00+05:30,9429.40,9450.00,9415.00,9441.55,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075
2020-05-05 09:20:00+05:30,9441.35,9448.70,9416.40,9417.15,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075
2020-05-05 09:25:00+05:30,9416.75,9416.75,9388.20,9391.60,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075
2020-05-05 09:30:00+05:30,9391.60,9411.80,9380.00,9399.40,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075
2020-05-05 09:35:00+05:30,9399.75,9422.05,9397.15,9418.00,0,2020-05-05,9429.4,9450.9,9190.75,9205.6,9533.5,9533.5,9266.95,9293.5,9364.65,9400.225,9329.075


In [527]:
df_5min['close_1per'] = df_5min['prev_close'] * .001
df_5min['tc-bc'] = abs(df_5min['top_pivot'] - df_5min['bottom_pivot'])
df_5min['cpr_range']  = np.where(df_5min['tc-bc'] < df_5min['close_1per'], "narrow", 'wide')

In [528]:
df_5min = df_5min.between_time('09:15', '15:10').copy()
# df_5min.loc['date_only'] = df_5min.index.date
df_5min['low_down'] = np.where(df_5min.index.time == pd.to_datetime('09:25:00').time(),df_5min['Low'] - 5, np.nan)
df_5min['high_up'] = np.where(df_5min.index.time == pd.to_datetime('09:25:00').time(),df_5min['High'] - 5, np.nan)
df_5min.ffill(inplace=True)
df_5min.index = df_5min.index.tz_localize(None)
df_5min.dropna(inplace = True)

In [529]:
df_5min.tail()

,Open,High,Low,Close,Volume,date_only,day_open,day_high,day_low,day_close,...,prev_low,prev_close,pivot,bottom_pivot,top_pivot,close_1per,tc-bc,cpr_range,low_down,high_up
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-20 14:50:00,18042.05,18056.00,18041.55,18047.55,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25
2023-01-20 14:55:00,18046.75,18049.50,18038.85,18049.40,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25
2023-01-20 15:00:00,18050.40,18054.25,18032.90,18035.25,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25
2023-01-20 15:05:00,18035.30,18037.70,18020.05,18030.95,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25
2023-01-20 15:10:00,18031.15,18032.90,18024.75,18029.35,0,2023-01-20,18115.6,18145.45,18016.2,18027.65,...,18063.75,18107.85,18108.933333,18109.475,18108.391667,18.10785,1.083333,narrow,18071.35,18103.25


In [533]:
def low_high(data):
    # print(data)
    return data

class nifty_3candle(Strategy): 
    flag = 0
    date_list = []
    def init(self):
        self.low_down = self.I(low_high, self.data.low_down, plot = False)
        self.high_up = self.I(low_high, self.data.high_up , plot = False)
    
    def next(self):
        # print(self.data.index.date)
        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime('09:25:00').time()):
            self.flag = 0
            self.date_list.append(self.data.index.date[-1])
        

        if self.data.index[-1].time() == pd.to_datetime('15:05:00').time():
            self.position.close()
        
        
        if self.data.index[-1].time() > pd.to_datetime('09:25:00').time():
            if self.data.Close[-1] < self.low_down[-1] and not self.position and self.flag == 0 and self.data.cpr_range[-1] == 'wide':
                self.sell(sl = self.high_up[-1])
                self.flag = 1
            elif self.data.Close[-1] > self.high_up[-1] and not self.position and self.flag == 0  and self.data.cpr_range[-1] == 'wide':
                self.buy(sl = self.low_down[-1])
                self.flag = 1

bt = Backtest(df_5min,nifty_3candle, cash = 100000)

In [534]:
stats = bt.run()
stats

Start                     2020-05-05 09:25:00
End                       2023-01-20 15:10:00
Duration                    990 days 05:45:00
Exposure Time [%]                   36.945703
Equity Final [$]                     120776.8
Equity Peak [$]                      121047.7
Return [%]                            20.7768
Buy & Hold Return [%]               91.973146
Return (Ann.) [%]                    7.413989
Volatility (Ann.) [%]                7.396464
Sharpe Ratio                         1.002369
Sortino Ratio                        2.538889
Calmar Ratio                         1.413355
Max. Drawdown [%]                   -5.245665
Avg. Drawdown [%]                   -0.375521
Max. Drawdown Duration      269 days 21:50:00
Avg. Drawdown Duration        5 days 14:37:00
# Trades                                  513
Win Rate [%]                        29.239766
Best Trade [%]                       2.679942
Worst Trade [%]                     -1.048326
Avg. Trade [%]                    

In [475]:
# stats._equity_curve.plot()

In [532]:
bt.plot(resample = False)

Row(id='21891', ...)

In [510]:
df_trades = stats._trades

In [523]:
flag_addup = 0
cons_list = []
prev_val =df_trades['PnL'][0]
for i in df_trades['PnL']:
    if prev_val > 0 and i < 0:
        flag_addup += 1
        cons_list.append(flag_addup)
    elif prev_val < 0 and i > 0:
        flag_addup += 1
        cons_list.append(flag_addup)
    else:
        cons_list.append(flag_addup)
    prev_val = i 
df_trades['flag_addup'] = cons_list
df_trades['entry_date'] = df_trades['EntryTime'].dt.date
        

In [522]:
df_trades[(df_trades['entry_date'] > pd.to_datetime("2020-09-01")) & (df_trades['entry_date'] < pd.to_datetime("2021-02-27"))]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,flag_addup,year,month,day_name,entry_date
67,8,6843,6865,11557.75,11541.90,-126.8,-0.001371,2020-09-03 09:40:00,2020-09-03 11:30:00,0 days 01:50:00,24,2020,9,Thursday,2020-09-03
68,8,6915,6975,11371.75,11345.90,-206.8,-0.002273,2020-09-04 09:40:00,2020-09-04 14:40:00,0 days 05:00:00,24,2020,9,Friday,2020-09-04
69,-8,6986,7007,11309.45,11338.70,-234.0,-0.002586,2020-09-07 09:35:00,2020-09-07 11:20:00,0 days 01:45:00,24,2020,9,Monday,2020-09-07
70,8,7059,7122,11359.95,11338.45,-172.0,-0.001893,2020-09-08 09:40:00,2020-09-08 14:55:00,0 days 05:15:00,24,2020,9,Tuesday,2020-09-08
71,8,7130,7133,11266.75,11244.85,-175.2,-0.001944,2020-09-09 09:35:00,2020-09-09 09:50:00,0 days 00:15:00,24,2020,9,Wednesday,2020-09-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,6,16059,16105,15058.20,15025.75,-194.7,-0.002155,2021-02-19 09:40:00,2021-02-19 13:30:00,0 days 03:50:00,62,2021,2,Friday,2021-02-19
156,-6,16135,16197,14926.80,14699.00,1366.8,0.015261,2021-02-22 10:00:00,2021-02-22 15:10:00,0 days 05:10:00,63,2021,2,Monday,2021-02-22
157,-6,16202,16210,14733.25,14767.75,-207.0,-0.002342,2021-02-23 09:35:00,2021-02-23 10:15:00,0 days 00:40:00,64,2021,2,Tuesday,2021-02-23
158,6,16274,16352,14789.80,15106.10,1897.8,0.021386,2021-02-24 09:35:00,2021-02-25 15:10:00,1 days 05:35:00,65,2021,2,Wednesday,2021-02-24


In [517]:
df_trades[df_trades.flag_addup == 96]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,flag_addup,year,month,day_name
248,-5,26582,26583,17261.20,17286.40,-126.00,-0.001460,2021-09-03 09:40:00,2021-09-03 09:45:00,0 days 00:05:00,96,2021,9,Friday
249,5,26653,26662,17416.70,17379.55,-185.75,-0.002133,2021-09-06 09:35:00,2021-09-06 10:20:00,0 days 00:45:00,96,2021,9,Monday
250,-5,26869,26878,17315.80,17342.70,-134.50,-0.001553,2021-09-09 09:35:00,2021-09-09 10:20:00,0 days 00:45:00,96,2021,9,Thursday
251,-5,26947,26976,17286.40,17335.55,-245.75,-0.002843,2021-09-13 10:05:00,2021-09-13 12:30:00,0 days 02:25:00,96,2021,9,Monday
252,5,27013,27016,17435.80,17409.55,-131.25,-0.001506,2021-09-14 09:35:00,2021-09-14 09:50:00,0 days 00:15:00,96,2021,9,Tuesday
253,5,27157,27160,17563.85,17548.55,-76.50,-0.000871,2021-09-16 09:35:00,2021-09-16 09:50:00,0 days 00:15:00,96,2021,9,Thursday
254,-5,27230,27231,17714.00,17735.15,-105.75,-0.001194,2021-09-17 09:40:00,2021-09-17 09:45:00,0 days 00:05:00,96,2021,9,Friday
255,5,27301,27356,17536.30,17462.05,-371.25,-0.004234,2021-09-20 09:35:00,2021-09-20 14:10:00,0 days 04:35:00,96,2021,9,Monday
256,5,27375,27378,17463.80,17410.35,-267.25,-0.003061,2021-09-21 09:45:00,2021-09-21 10:00:00,0 days 00:15:00,96,2021,9,Tuesday
257,5,27449,27468,17579.90,17534.80,-225.50,-0.002565,2021-09-22 09:55:00,2021-09-22 11:30:00,0 days 01:35:00,96,2021,9,Wednesday


In [514]:
df_trades['year']  = df_trades['EntryTime'].dt.year
df_trades['month']  = df_trades['EntryTime'].dt.month
df_trades['day_name']  = df_trades['EntryTime'].apply(lambda x: datetime.strftime(x, '%A'))
pd.pivot_table(df_trades, index = "year", columns = ['day_name'], values = "PnL",aggfunc= sum)

day_name,Friday,Monday,Thursday,Tuesday,Wednesday
year,,,,,
2020,-2239.10,351.15,1310.95,-2989.9,2408.15
2021,1931.45,3445.50,2787.25,-2949.4,-1267.20
2022,-2605.50,7795.85,5022.20,807.5,3689.80
2023,NaN,1189.50,-663.30,1253.4,1498.50


## fibonacci

In [ ]:
# 1. calculate the diffrence of high and low of first candle
# 2. multiply the diffrence with [0.312,.618,1.618,2.618]
# 3. ADD the above value to low_high and subtract  lower value from low
# 4. buy and sell  when level breaches  .618 either upside or downside
# 5. target 1 is 1.618
# 6. sl is .312( very tight), high or low either side

In [306]:
start_dt = '2019-01-01'
end_dt = '2023-02-02'
time_frame = '5minute'
instument =256265 #256265 # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/49 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/49 [00:00<?, ?it/s]

In [307]:
df_day_fetch = temp1.copy()
df_5min = temp2.copy()

In [308]:
df_5min['first_candle_diff'] = np.where(df_5min.index.time == pd.to_datetime('09:15:00').time(),df_5min['High'] - df_5min['Low'] , np.nan)
df_5min['first_candle_high'] = np.where(df_5min.index.time == pd.to_datetime('09:15:00').time(),df_5min['High'] , np.nan)
df_5min['first_candle_low'] = np.where(df_5min.index.time == pd.to_datetime('09:15:00').time(),df_5min['Low'] , np.nan)
df_5min.ffill(inplace=True)
df_5min = df_5min.between_time('09:15', '15:20').copy()
df_5min.index = df_5min.index.tz_localize(None)
# df_5min.dropna(inplace = True)

In [ ]:
def low_high(data):
    # print(data)
    return data

def fib_indicator(high,first_candle, target):
    return (high + (first_candle* target))

def fib_indicator_below(low,first_candle, target):
    return (low  - (first_candle* target))  
    
class fibonacci_first(Strategy): 
    flag = 0
    date_list = []

    break_level = 0.618
    target_level = 1.618
    start_candle  = '09:15:00'

    def init(self):
        # indicator of buy
        self.break_level_above = self.I(fib_indicator, self.data.first_candle_high,self.data.first_candle_diff,self.break_level)
        self.target_level_above = self.I(fib_indicator, self.data.first_candle_high,self.data.first_candle_diff,self.target_level)
        self.sl_level_above = self.I(low_high,self.data.first_candle_low, plot = False)
        # indicator for sell
        self.break_level_below = self.I(fib_indicator_below, self.data.first_candle_low,self.data.first_candle_diff,self.break_level)
        self.target_level_below = self.I(fib_indicator_below, self.data.first_candle_low,self.data.first_candle_diff,self.target_level)
        self.sl_level_below = self.I(low_high,self.data.first_candle_high, plot = False)

        
    def next(self):
        # getting 1 trade a day
        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
            self.flag = 3
            self.date_list.append(self.data.index.date[-1])

        if self.data.index[-1].time() == pd.to_datetime('15:20:00').time():
            self.position.close()
        try:
            # print(self.sl_level_below,self.break_level_below,self.target_level_below)
            if self.break_level_above[-1] < self.data.Close[-1] and self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and not self.position and self.flag > 0:
                self.buy(sl = self.sl_level_above[-1], tp = self.target_level_above[-1])
                self.flag -= 1
                
            if self.break_level_below[-1] > self.data.Close[-1] and \
                self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and self.flag > 0 \
                and not self.position:
                # print(self.data)
                self.sell(sl = self.sl_level_below[-1], tp = self.target_level_below[-1])
                self.flag -= 1
        except:
            # print("try",self.data)
            # print('try')
            pass

bt = Backtest(df_5min,fibonacci_first, cash = 100000)

In [314]:
stats = bt.run()
stats

In [255]:
df_trades = stats._trades
df_trades['entry_date'] = df_trades['EntryTime'].dt.date
df_trades['entry_date'].value_counts()

2022-02-28    3
2022-01-20    3
2023-01-06    3
2022-01-05    2
2022-06-10    2
             ..
2022-05-25    1
2022-05-13    1
2022-05-11    1
2022-05-05    1
2023-01-20    1
Name: entry_date, Length: 98, dtype: int64

In [256]:
df_trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,entry_date
0,5,14,18,17820.10,17931.4484,556.7420,0.006248,2022-01-05 09:15:00,2022-01-05 13:15:00,0 days 04:00:00,2022-01-05
1,5,20,21,17935.40,17768.5000,-834.5000,-0.009306,2022-01-05 15:15:00,2022-01-06 09:15:00,0 days 18:00:00,2022-01-05
2,5,55,56,18215.10,18257.0000,209.5000,0.002300,2022-01-12 15:15:00,2022-01-13 09:15:00,0 days 18:00:00,2022-01-12
3,-5,83,84,18096.55,18129.2000,-163.2500,-0.001804,2022-01-18 15:15:00,2022-01-19 09:15:00,0 days 18:00:00,2022-01-18
4,-5,95,95,17769.80,17680.7467,445.2665,0.005011,2022-01-20 13:15:00,2022-01-20 13:15:00,0 days 00:00:00,2022-01-20
...,...,...,...,...,...,...,...,...,...,...,...
116,-6,1761,1761,17831.70,17803.9114,166.7316,0.001558,2023-01-06 13:15:00,2023-01-06 13:15:00,0 days 00:00:00,2023-01-06
117,-6,1762,1762,17820.60,17803.9114,100.1316,0.000936,2023-01-06 14:15:00,2023-01-06 14:15:00,0 days 00:00:00,2023-01-06
118,-6,1763,1764,17869.45,17952.5500,-498.6000,-0.004650,2023-01-06 15:15:00,2023-01-09 09:15:00,2 days 18:00:00,2023-01-06
119,6,1796,1799,17964.35,18033.1500,412.8000,0.003830,2023-01-13 13:15:00,2023-01-16 09:15:00,2 days 20:00:00,2023-01-13


In [257]:
bt.plot(resample=False)

c:\Users\akash.verma\Miniconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.
  """Entry point for launching an IPython kernel.


Row(id='19871', ...)

In [103]:
df_trades['days'] = df_trades.Duration.dt.days

In [104]:
df_trades[df_trades['days']> 0]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,entry_date,days
11,5,728,740,17967.8,18033.15,326.75,0.003637,2023-01-13 14:25:00,2023-01-16 09:15:00,2 days 18:50:00,2023-01-13,2


In [799]:

df_trades['year']  = df_trades['EntryTime'].dt.year
df_trades['month']  = df_trades['EntryTime'].dt.month
df_trades['day_name']  = df_trades['EntryTime'].apply(lambda x: datetime.strftime(x, '%A'))
pd.pivot_table(df_trades, index = "year", columns = ['month'], values = "PnL",aggfunc= sum)

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2022,1516.4200,483.4158,6110.3268,-1045.8624,-215.1204,1846.3574,2738.1384,2905.4556,3710.0412,2112.5454,-17.9346,1276.0614
2023,2005.4628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## MACD

In [188]:
import ta

In [210]:
start_dt = '2023-01-01'
end_dt = '2023-02-02'
time_frame = '60minute'
instument =256265 #256265 # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

In [211]:
df_day_fetch = temp1.copy()
df_5min = temp2.copy()

In [212]:
df_5min = df_5min.between_time('09:15', '15:20').copy()
df_5min.index = df_5min.index.tz_localize(None)

In [213]:

    
class Mcd_strategy(Strategy): 
    flag = 0
    date_list = []
    start_candle  = '09:15:00'

    def init(self):
        # indicator of buy
        close = self.data.Close
        self.macd = self.I(ta.trend.macd, pd.Series(close))
        self.macd_signal = self.I(ta.trend.macd_signal, pd.Series(close))
        self.ema_100 = self.I(ta.trend.ema_indicator, pd.Series(close), window = 100)

    def next(self):
        price = self.data.Close  

        if (self.data.index.date[-1] not in self.date_list) and (self.data.index[-1].time() == pd.to_datetime(self.start_candle).time()):
            self.flag = 3
            self.date_list.append(self.data.index.date[-1])
        
        elif self.data.index[-1].time() == pd.to_datetime('15:20:00').time():
            self.position.close()
         
        elif crossover(self.macd, self.macd_signal) and (price > self.ema_100) and not self.position and self.flag > 0:
            sl = price * .97
            tp = price * 1.045
            self.buy(sl = sl , tp = tp)
            self.flag -= 1
        # getting 1 trade a day


        # try:
        #     # print(self.sl_level_below,self.break_level_below,self.target_level_below)
        #     if self.break_level_above[-1] < self.data.Close[-1] and self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and not self.position and self.flag > 0:
        #         self.buy(sl = self.sl_level_above[-1], tp = self.target_level_above[-1])
        #         self.flag -= 1
                
        #     if self.break_level_below[-1] > self.data.Close[-1] and \
        #         self.data.index[-1].time() > pd.to_datetime(self.start_candle).time() and self.flag > 0 \
        #         and not self.position:
        #         # print(self.data)
        #         self.sell(sl = self.sl_level_below[-1], tp = self.target_level_below[-1])
        #         self.flag -= 1
        # except:
        #     # print("try",self.data)
        #     # print('try')
        #     pass

bt = Backtest(df_5min,Mcd_strategy, cash = 100000)

In [214]:
stats = bt.run()
stats

Start                     2023-01-02 09:15:00
End                       2023-01-20 15:15:00
Duration                     18 days 06:00:00
Exposure Time [%]                         0.0
Equity Final [$]                     100000.0
Equity Peak [$]                      100000.0
Return [%]                                0.0
Buy & Hold Return [%]               -0.746577
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [209]:
bt.plot()

Row(id='18559', ...)

## vector by

In [9]:
import vectorbt as vbt

In [11]:
start_dt = '2023-01-01'
end_dt = '2023-02-02'
time_frame = '5minute'
instument =256265 #256265 # 260105(bank) #341249(hdfc)
df_5min = uf.get_data_parllel(kite, instument, time_frame , start_dt,end_dt)
df_day_fetch = uf.get_data_parllel(kite, instument, 'day' , start_dt,end_dt)
temp1 = df_day_fetch.copy()
temp2 = df_5min.copy()

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
fast_ma = vbt.MA.run(df_5min['Close'], 10, short_name='fast')
slow_ma = vbt.MA.run(df_5min['Close'], 20, short_name='slow')
entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

In [25]:
fast_ma

AttributeError: 'MA' object has no attribute 'count'

In [21]:
pf = vbt.Portfolio.from_signals(df_5min['Close'], entries, exits)

In [22]:
pf.total_return()

0.004697263527415459